### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)

In [2]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product

conds = {'0':['good','bad','GoodBad'],'1':['poor','partial','EpiPerf'],'2':['partial','detailed','EpiPerf'],
         '3':['poor','detailed','EpiPerf']}
for cond in conds:
    ###############################################################################
    st = study('Olfacto')
    path_npz = join(st.path, 'figures_npz/1_'+conds[cond][0]+'_'+conds[cond][1]+'_4500_expi_noart/')
    path_to_save = join(path_npz, 'masks_stat/')
    f_form = '{}_sources_phase_{}_odor_'+conds[cond][0]+'_'+conds[cond][1]+'_Expi_sel_phys.npz'
    f_form = join(path_npz, f_form)
    f_save = '{}_mask_stat_{}_minwin{}_th{}.npy'
    f_save = join(path_to_save,f_save)
    ###############################################################################
    if not exists(path_to_save):
        makedirs(path_to_save)
    ###############################################################################  
    freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha']
    min_wins = [1,2,3]

    for freq, min_win in product(freqs, min_wins):
        # Load the npz file with su_codes, xyz of signif and not signif elec
        mat = np.load(f_form.format('All_subjects',freq))
        subjects,s_th05, s_da, s_th01 = mat['su_codes'],mat['s_th_05'], mat['s_da'],mat['s_th_01']
        #print(subjects.shape, s_th05.shape, s_th01.shape, s_da.shape)
        su_codes = np.unique(subjects)
        mask05 = np.array([])
        for su in su_codes:
            n_elecs = s_th05[np.where(subjects ==su)].shape[0]
            th05 = np.amax(s_th05[np.where(subjects==su)])
            da = s_da[np.where(subjects==su)]
            #print(n_elecs,da.shape,th05)
            for elec in range(n_elecs):
                da_elec = da[elec,:]
                pvals = np.array([])
                for win in range(da_elec.shape[0]):
                    if da_elec[win] > th05:
                        pval = 1
                    if da_elec[win] <= th05:
                        pval = 0
                    pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
                #criteria for an elec to be significant
                pvals = np.ravel(pvals)
                underp = np.where(pvals == 1)[0]
                pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
                signif = [True for k in pvsplit if len(k) >= min_win]
                if len(signif) >=1:
                    remove = False
                else:
                    remove = True
                mask05 = np.hstack((mask05, remove)) if np.size(mask05) else remove
        print(freq, len(mask05),len(np.where(mask05 == False)[0]))
        np.save(f_save.format('All_subjects',freq,min_win,'05'),mask05)

-> Olfacto loaded
0_VLFC 499 41
0_VLFC 499 10
0_VLFC 499 4
1_delta 499 45
1_delta 499 4
1_delta 499 0
2_theta 499 61
2_theta 499 5
2_theta 499 0
3_alpha 499 76
3_alpha 499 3
3_alpha 499 0
-> Olfacto loaded
0_VLFC 453 31
0_VLFC 453 9
0_VLFC 453 3
1_delta 453 46
1_delta 453 2
1_delta 453 0
2_theta 453 58
2_theta 453 1
2_theta 453 1
3_alpha 453 70
3_alpha 453 5
3_alpha 453 1
-> Olfacto loaded
0_VLFC 607 72
0_VLFC 607 15
0_VLFC 607 5
1_delta 607 72
1_delta 607 10
1_delta 607 0
2_theta 607 87
2_theta 607 3
2_theta 607 0
3_alpha 607 89
3_alpha 607 5
3_alpha 607 0
-> Olfacto loaded
0_VLFC 453 62
0_VLFC 453 12
0_VLFC 453 2
1_delta 453 72
1_delta 453 4
1_delta 453 0
2_theta 453 42
2_theta 453 0
2_theta 453 0
3_alpha 453 59
3_alpha 453 3
3_alpha 453 1


### 2- Create masks - min patients

In [8]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product
from PIL import Image
score = 'Epi'

conds = ['partial','detailed']
###############################################################################
st = study('Olfacto')
path_classif = join(st.path, 'classified/1_Classif_Phase_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_EpiPerf_4500_expi_noart/')
path_npz = join(st.path, 'figures_npz/1_'+conds[0]+'_'+conds[1]+'_4500_expi_noart/')
path_rois, path_stats = join(path_npz,'Signif_ROIs/'), join(path_npz, 'masks_stat/')
path_to_save = join(path_npz, 'masks_visbrain/')
path_save_fig = join(path_npz, 'Signif_timecourse/')
npz_form = join(path_npz, '{}_sources_phase_{}_odor_'+conds[0]+'_'+conds[1]+'_Expi_sel_phys.npz')
rois_form = join(path_rois, '{}_signif_min_{}_{}_min_win{}_th_{}.npy')
stat_form = join(path_stats,'{}_mask_stat_{}_minwin{}_th{}.npy')
mask_vis_form = join(path_to_save,'{}_mask_min{}_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
if not exists(path_save_fig):
    makedirs(path_save_fig)
############################################################################### 

freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha'] #'0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2
su_codes = {'S0':'LEFC','S1':'CHAF','S2':'VACJ','S3':'SEMC','S4':'FERJ','S5':'MICP','S6':'PIRJ'}
min_wins = ['1','2','3']
bsl='None'
methods = ['aal_RL','BA'] #,'aal_RL','BA','aal',
th ='05'
min_sigs = [2,3,4,5]

for freq,min_win,min_sig,method in product(freqs,min_wins,min_sigs,methods):
    if exists(rois_form.format(method,min_sig,freq[2:],min_win,th)):
        # Load the electrodes labels, electrodes significant (bool), rois significant (names)
        npz_mat = np.load(npz_form.format('All_subjects',freq))
        print(npz_mat.files)
        labels0, labels, subjects = npz_mat['s_labels'],npz_mat['s_'+method],npz_mat['su_codes']
        elecs, channels = npz_mat['s_elec'],npz_mat['s_channels']
        stats = np.load(stat_form.format('All_subjects',freq,min_win,th))
        rois = np.load(rois_form.format(method,min_sig,freq[2:],min_win,th))
        #print(rois)
        mask_vis = []
        i=0
        for e in range(labels.shape[0]):
            if stats[e]==0 and labels[e] in rois:
                mask_vis.append(False)
                i=i+1
                print(subjects[e],labels[e],channels[e],elecs[e],'number of elecs',i)
                #path_file = path_classif+su_codes[subjects[e]]+'/'+freq+'/fig/'
                path_file_time = path_classif+freq+'/fig/'
                filename = su_codes[subjects[e]]+'_Phase_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+channels[e]+'_'+labels0[e]+'_('+elecs[e]+').png'
                img = Image.open(path_file_time+filename)
                path_save_fig2 = path_save_fig+freq+'/'+method+'/'+labels[e]+'/'
                if not exists(path_save_fig2):
                    makedirs(path_save_fig2)
                img.save(path_save_fig2+'/min_patient'+str(min_sig)+'_minwin_'+str(min_win)+'_'+filename,'png')
            else :
                mask_vis.append(True)
            #print(stats[e],labels[e], mask_vis[e])
        np.save(mask_vis_form.format(method,min_sig,freq,min_win,th),mask_vis)
    else:
        print('ça marche pas')
        labels = np.load(npz_form.format('All_subjects',freq))['s_'+method]
        mask_vis = True*np.ones(len(labels))
        np.save(mask_vis_form.format(method,min_sig,freq,min_win,th),mask_vis)
    #print('»»»', freq, win, min_sig, method)
    #[0'LEFC',1'CHAF',2'VACJ',3'SEMC',4'FERJ',5'MICP',6'PIRJ']


-> Olfacto loaded
['s_labels', 's_aal_RL', 's_channels', 's_th_01', 's_da', 's_xyz', 's_aal', 's_BA', 's_th_05', 's_elec_p1', 'su_codes', 's_elec_p0', 's_time', 's_elec']
S0 Cingulum Ant k2-k1 36 number of elecs 1
S0 Cingulum Ant k4-k3 38 number of elecs 2
S0 Frontal Mid s9-s8 96 number of elecs 3
S1 Frontal Mid g'6-g'5 59 number of elecs 4
S1 Frontal Mid h'10-h'9 70 number of elecs 5
S2 Temporal Mid b8-b7 3 number of elecs 6
S2 Temporal Mid b10-b9 5 number of elecs 7
S2 Temporal Mid d'10-d'9 19 number of elecs 8
S2 Insula e'3-e'2 23 number of elecs 9
S2 Cingulum Ant k2-k1 32 number of elecs 10
S2 Frontal Mid k12-k11 37 number of elecs 11
S5 Insula t'2-t'1 63 number of elecs 12
S5 Temporal Mid y'3-y'2 70 number of elecs 13
['s_labels', 's_aal_RL', 's_channels', 's_th_01', 's_da', 's_xyz', 's_aal', 's_BA', 's_th_05', 's_elec_p1', 'su_codes', 's_elec_p0', 's_time', 's_elec']
S0 BA21 a8-a7 3 number of elecs 1
S0 BA20 d9-d8 20 number of elecs 2
S0 BA32 k2-k1 36 number of elecs 3
S0 BA32 k4